In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# #importing all the packages
# import numpy as np 
# import pandas as pd

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [3]:
# import BERT tokenization
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [4]:
#reading a tsv file containing training and testing data
#load data
train=pd.read_csv('/kaggle/input/training/simpsons_dataset-training.tsv',sep='\t')
test=pd.read_csv('/kaggle/input/testing/simpsons_dataset-testing.tsv',sep='\t')
submission= pd.read_csv('/kaggle/input/submission/simpsons_dataset-sample_submission.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/training/simpsons_dataset-training.tsv'

In [99]:
pip install tokenization

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [100]:
pip install bert-for-tf2

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [101]:
#Preprocessing
#importing packages
#tokenization takes raw data and converts it into a useful data string
from bert import bert_tokenization
import tokenization
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [102]:
#label encoding 
label = preprocessing.LabelEncoder()
y = label.fit_transform(train['class'])
y = to_categorical(y)
print(y[:5])

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]


In [103]:
#building a BERT layer
m_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
#wraps a callable object
bert_layer = hub.KerasLayer(m_url, trainable=True)

KeyboardInterrupt: 

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train['Category'] = encoder.fit_transform(train['class'])

In [ ]:
#encoding the text
#numpy array
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
#changing the text to lowercase
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
#passing the vocab_file and do_lower_case to the tokenization
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    for text in texts:
        text = tokenizer.tokenize(text)
        
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len-len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
#bert model
#Keras API
#Three dense neural network layers and a dropout layer
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    
    clf_output = sequence_output[:, 0, :]
    lay = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    lay = tf.keras.layers.Dense(32, activation='relu')(lay)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    out = tf.keras.layers.Dense(5, activation='softmax')(lay)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    # using categorical crossentropy as the loss
    model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
#checking first 250 characters of each text
max_len = 250
train_input = bert_encode(train.text.values, tokenizer, max_len=max_len)
test_input = bert_encode(test.text.values, tokenizer, max_len=max_len)
train_labels = y

In [ ]:
labels = label.classes_
print(labels)

In [106]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 250)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 250)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 250)]        0                                            
__________________________________________________________________________________________________
keras_layer_4 (KerasLayer)      [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [107]:
#run the model
#training for 3 epochs
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
train_sh = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=10,
    callbacks=[checkpoint, earlystopping],
    batch_size=24,
    verbose=1
)



Epoch 1/10


KeyboardInterrupt: 

In [108]:
#Evaluation
model.load_weights('model.h5')
#making predictions
preds = model.predict(test_input,verbose = 1)
#converting the one hot vector output to a linear numpy array.
pred_classes = np.argmax(preds, axis = 1)

#mapping the encoded output to actual categories
predicted_category = [labels[x] for x in pred_classes]

submission['Category'] = predicted_category
submission.to_csv('submission.csv', index=False)
display(submission.head(10))

329/329 [==============================] - 105s 318ms/step


,Id,Category
0,1,Bart Simpson
1,2,Lisa Simpson
2,3,Homer Simpson
3,4,Homer Simpson
4,5,Homer Simpson
5,6,Homer Simpson
6,7,Homer Simpson
7,8,Homer Simpson
8,9,Other
9,10,Marge Simpson


In [105]:
#final check
submission.shape

(10499, 2)